In Stommel model, use the values rtol=1e-4, atol=1e-8
Plot T-S in addition to t,s etc.
Use 0.9 for sigma_zero (based on the bifurcation diagram, basically freshwater flux)
Use temp=1 and salinity=0.5 for AMOC on
Do long simulation til it ends transient phase, cut transient, compute AC function of time series, compute variance (transient may also be equal to AC time)
Find AC function on time, checking correlation between quantity and itself at lag
Can either
Have a very long run, split time series in blocks (for example, 100 blocks 100 timesteps long in 10k years), compute expectation inside brackets, where t0 is first value of time series, E becomes average over 100 windows of split time series

In [35]:
import numpy as np
import pandas as pd
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
import matplotlib

In [38]:
def smoothabs(x, xi=10000):
    """Smooth absolute value function"""
    return x * np.tanh(x * xi)

In [39]:
def gottwald_noice(t, u, p, stochsys=True):
    """Gottwald model functions without sea ice"""
    x, y, z, T, S = u
    if stochsys:
        pvec = p[0] if isinstance(p, list) else p
    else:
        pvec = p
    
    (a, b, mu, epsilon_a, epsilon_f, F0, F1, G0, G1, 
     theta_0, theta_1, sigma_0, sigma_1, x_mean, Delta_mean) = pvec
    
    Delta = y**2 + z**2
    T_surf = theta_0 + theta_1 * (x - x_mean)/(np.sqrt(epsilon_f))
    S_surf = sigma_0 + sigma_1 * (Delta - Delta_mean)/(np.sqrt(epsilon_f))
    
    dx = 1/epsilon_f*(-Delta - a*(x - F0 - F1*T))
    dy = 1/epsilon_f*(x*y - b*x*z - (y - G0 + G1*T))
    dz = 1/epsilon_f*(b*x*y + x*z - z)
    dT = -1/epsilon_a*(T - T_surf) - T - mu*smoothabs(S-T)*T
    dS = S_surf - S - mu*smoothabs(S-T)*S
    
    return np.array([dx, dy, dz, dT, dS])

In [40]:
def param_gwn_default(param_l84=None, sigma_0=0.9): #**kwargs):
    """return parameter values"""
    if param_l84 is not None:
        a, b, F0, G0 = param_l84
    else:
        param_l84 = [0.25, 4., 8., 1.]  # default L84 params [a, b, F0, G0]
        
    x_std = 0.513
    Delta_std = 1.071
    
    x_mean = 1.0147
    Delta_mean = 1.7463  # long-run results from default L84
    
    # Unpack L84 params
    a, b, F0, G0 = param_l84
    
    # Other default model params
    mu = 7.5
    theta_0 = 1.
    # sigma_0 = 0.9  # Stommel params (is varied) 0.926, 0.932
    epsilon_a = 0.34
    epsilon_f = 0.0003  # 0.0083 # timescales
    F1 = 0.1
    G1 = 0.  # coupling params
    perturb_scaling = 0.01  # coupling strength of L84 to Stommel
    
    # Coupling params from L84 run/default
    # theta_1 = min(theta_0, perturb_scaling/x_std)
    # sigma_1 = min(sigma_0, perturb_scaling/Delta_std)

    theta_1=0.0195
    sigma_1=0.00934
    
    return [a, b, mu, epsilon_a, epsilon_f, F0, F1, G0, G1, 
            theta_0, theta_1, sigma_0, sigma_1, x_mean, Delta_mean]


In [41]:
def simulate_gottwald_noice(initial_conditions, params, t_span, t_eval=None, stochsys=True):
    """Simulate the Gottwald model without sea ice"""
    sol = solve_ivp(
        lambda t, y: gottwald_noice(t, y, params, stochsys),
        t_span,
        initial_conditions,
        t_eval=t_eval,
        method='RK45',
        rtol=1e-4,
        atol=1e-8
    )
    return sol

In [42]:
#control run only
if __name__ == "__main__":
    matplotlib.use('Agg')
    
    # Get default Gottwald parameters
    params_gwn = param_gwn_default()
    print(f"Default Gottwald parameters:")
    print(params_gwn)

    # params_gwn_wrapped = [params_gwn]  # Wrap in list for stochsys format
    tmax = 5010 # 00
    t_eval = np.linspace(0, tmax, tmax*100)

    T0=0.5
    S0=0.5
    initial_gwn = [1.0, 0.5, 0.5, T0, S0]  # [x, y, z, T, S]
        
    sol_gwn = simulate_gottwald_noice(initial_gwn, params_gwn, (0, tmax), 
                                   t_eval=t_eval, stochsys=False)
    data = {
        't': sol_gwn.t,
        'x': sol_gwn.y[0],
        'y': sol_gwn.y[1],
        'z': sol_gwn.y[2],
        'T': sol_gwn.y[3],
        'S': sol_gwn.y[4],
        'S0': S0,
        'T0': T0,
        'AMOC': sol_gwn.y[3]-sol_gwn.y[4]
    }

    control_run = pd.DataFrame(data)

Default Gottwald parameters:
[0.25, 4.0, 7.5, 0.34, 0.0003, 8.0, 0.1, 1.0, 0.0, 1.0, 0.0195, 0.9, 0.00934, 1.0147, 1.7463]


In [43]:
control_run.head()
control_run.to_csv('control_run5000_year.csv',index=False)

In [17]:
'''
The changes here are that the AMOC index is computed. Pairs of T and S are looped through to change initial conditions for
the solver. 
'''
# Example usage
all_runs=[]
if __name__ == "__main__":
    matplotlib.use('Agg')
    
    # Get default Gottwald parameters
    params_gwn = param_gwn_default()
    print(f"Default Gottwald parameters:")
    print(params_gwn)

    # params_gwn_wrapped = [params_gwn]  # Wrap in list for stochsys format
    tmax = 1 # 00
    t_eval = np.linspace(0, tmax, tmax*100)
    amoc=np.zeros((2,2,tmax*100))
    
    # Simulate Gottwald model, I change to one loop and df instead of array
    salts=[0.5,1,0.5,1]
    temps=[0.5,0.5,1,1]

    for temp in range(len(temps)):
        T0=salts[temp]
        S0=temps[temp]
        initial_gwn = [1.0, 0.5, 0.5, T0, S0]  # [x, y, z, T, S]
            
        sol_gwn = simulate_gottwald_noice(initial_gwn, params_gwn, (0, tmax), 
                                       t_eval=t_eval, stochsys=False)
        data = {
            't': sol_gwn.t,
            'T': sol_gwn.y[3],
            'S': sol_gwn.y[4],
            'S0,T0': f"{S0},{T0}",
            'AMOC': sol_gwn.y[3]-sol_gwn.y[4]
        }
        
        #Convert this run to a dataframe
        run_df = pd.DataFrame(data)
    
        #Store it
        all_runs.append(run_df)

#After the loop, combine everything:
amoc_df = pd.concat(all_runs, ignore_index=True)
        
'''for iT in range(2):
        for iS in range(2):
            #0.5, 0.5, 1, 1
            T0=(iT+1)*0.5
            print('T0:',T0)
            #0.5,1,0.5,1
            S0=(iS+1)*0.5
            print('S0:',S0)
            initial_gwn = [1.0, 0.5, 0.5, T0, S0]  # [x, y, z, T, S]
            
            sol_gwn = simulate_gottwald_noice(initial_gwn, params_gwn, (0, tmax), 
                                       t_eval=t_eval, stochsys=False)
            #print(iT,iS)
            amoc[iT,iS,:]=sol_gwn.y[3]-sol_gwn.y[4]
            print((amoc[iT,iS,:]))'''
amoc_df.head()

Default Gottwald parameters:
[0.25, 4.0, 7.5, 0.34, 0.0003, 8.0, 0.1, 1.0, 0.0, 1.0, 0.0195, 0.9, 0.00934, 1.0147, 1.7463]


,t,T,S,"S0,T0",AMOC
0,0.000000,0.500000,0.500000,"0.5,0.5",0.000000
1,0.010101,0.509444,0.503989,"0.5,0.5",0.005456
2,0.020202,0.518119,0.507743,"0.5,0.5",0.010375
3,0.030303,0.526034,0.511252,"0.5,0.5",0.014782
4,0.040404,0.532228,0.514728,"0.5,0.5",0.017500


In [18]:
amoc_df.to_csv('amoc_4traj_100y_block1.csv',index=False)

In [19]:
print(amoc_df['S0,T0'].unique())

['0.5,0.5' '0.5,1' '1,0.5' '1,1']


In [107]:
fig, ax = plt.subplots(1, 3, figsize=(4 * 2.5, 3))
for iT in range(2):
    for iS in range(2):
        ax[0].plot(sol_gwn.t, amoc[iT,iS,:])
ax[0].set_xlabel("t")
ax[0].set_ylabel("T-S")
ax[1].plot(sol_gwn.y[0][:1000], sol_gwn.y[1][:1000])
ax[1].plot(sol_gwn.y[0][-1000:], sol_gwn.y[1][-1000:])
ax[1].set_xlabel("x")
ax[1].set_ylabel("y")
ax[2].plot(sol_gwn.y[1][:1000], sol_gwn.y[2][:1000])
ax[2].plot(sol_gwn.y[1][-1000:], sol_gwn.y[2][-1000:])    
ax[2].set_xlabel("y")
ax[2].set_ylabel("z")
fig.tight_layout()   
fig.savefig(f"Gottwald.png")

/var/folders/v1/ymptnwwx3x1b7_mw0nns09wr0000gn/T/ipykernel_54646/1562123325.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots(1, 3, figsize=(4 * 2.5, 3))


In [89]:
print(len(sol_gwn.t))
print(len(amoc[iT,iS]))
print(tmax)
print(len(sol_gwn.t[:tmax*90]))
print(len(amoc[iT,iS,tmax*10:]))

100000
100000
1000
90000
90000


In [90]:
#this figure takes out the transients
fig, ax = plt.subplots(1, 3, figsize=(4 * 2.5, 3))
for iT in range(2):
    for iS in range(2):
        ax[0].plot(sol_gwn.t[:tmax*90], amoc[iT,iS,tmax*10:])
ax[0].set_xlabel("t")
ax[0].set_ylabel("T-S")
ax[1].plot(sol_gwn.y[0][:1000], sol_gwn.y[1][:1000])
ax[1].plot(sol_gwn.y[0][-1000:], sol_gwn.y[1][-1000:])
ax[1].set_xlabel("x")
ax[1].set_ylabel("y")
ax[2].plot(sol_gwn.y[1][:1000], sol_gwn.y[2][:1000])
ax[2].plot(sol_gwn.y[1][-1000:], sol_gwn.y[2][-1000:])    
ax[2].set_xlabel("y")
ax[2].set_ylabel("z")
fig.tight_layout()   
fig.savefig('Gottwald_no_transient.png')